In [1]:
# RUN THIS CELL FOR FORMAT
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

# Documentation
@merlionctc

**Table of Contents**
- [1. Introduction](#1.-Introduction)
  - [1.1 Derivative](#1.1-Derivatives)
  - [1.2 Automatic Differentiation](#1.2-Auto-Differentiation)
- [2. Background](#2.-Background)
  - [2.1 Chain Rule](#2.1-Chain-Rule) 
  - [2.2 Elementary functions](#2.2-Elementary-functions) 
  - [2.3 Forward mode](#2.3-Forward-mode) 
  - [2.4 Reversed mode](#2.4-Reversed-mode) 
  - [2.5 Dual Number](#2.5-Dual-Number) 
- [3. Usage](#3.-Usage)
  - [3.1 Installation](#3.1-Installation)
  - [3.2 How to use](#3.2-How-to-Use)
  - [3.3 Notes](#3.3-Notes)
- [4. Software organization](#4.-Software-Organization)
- [5. Implementation](#5.-Implementation)
- [6. Extensions](#6.-Extensions)
- [7. Broader Impact and Inclusivity Statement](#7.-Broader-Impact-and-Inclusivity-Statement)
- [8. Reference](#8.-Reference)





## 1. Introduction

We developed this package, `AutoDiff`,  in the light of automatic differentiation. The package can help to automatically differentiate a function input into the program. The package includes modules of forward-mode differentiation and backward-mode differentiation.

### 1.1 Derivatives

Formally, for single variable case, the derivative of a function, if it exists, is defined as

$$ f'(x) = \lim_{h\to0} \frac{f(a+h) - f(a)}{h} $$

to visualize, is the slope of the tangent line to the graph of the function at that point. The tangent line is the best linear approximation of the function near that input value.

Of course, derivatives may be generalized to functions of several real variables, and derivatives are useful in finding the maxima and minima of functions. Derivatives has a variety of applications in statistics and machine learning, and the process of finding a derivative is called *differentiation*.

There are three ways of differentiation realized in computer science:
- Numerical differentiation
- Symbolic differentiation
- Automatic differentiaton


### 1.2 Auto Differentiation
Automatic differentiation (AD), also called as algorithmic differentiation, is a set of techniques for efficiently and accurately evaluating derivatives of numeric functions expressed as computer programs <sup>1</sup>. It is not numerical differentiation, since numerical differentiation is the finite difference approximation of derivatives using the values of the original function evaluated at some sample points<sup>2</sup>. It is different from symbolic differentiation, since symbolic differentiation is the automatic manipulation of expressions for obtaining derivative expression<sup>3</sup>.

The essence of AD is that all numerical computations are ultimately compositions of a finite set of elementary operations for which the derivatives are easily known<sup>4</sup>. The algorithm of AD breaks down a function by looking at the sequence of elementary arithmetic operations (addition, subtraction, multiplication and division) and elementary functions (exponential, logrithmatic, and trigonometry). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to machine accuracy.

This differentiation technique well-established and used with applications in different areas such as fluid dynamics, astronomy, and engineering design optimization.

To sum up, there are two major advantages of using AD:
- Computes derivatives to machine precision.
- Does not rely on extensive mathematical derivations or expression trees, so it is easily applicable to a wide class of functions.

## 2. Background

Automatic differentiation relies on several vital mathematical foundations, some of which will be illustrated at this part. Based on these conceptions, it will be more resonable for users to understand the software.

### 2.1 Chain Rule

Chain Rule is the most important concept in AD. It enables us to deal with complex functions with several layers and arguments. With implementing chain rule, we can easily divide the original complicated functions into basic parts made up with elementary functions, of which we will know the concrete derivative expressions.

Suppose there is a function $h\left(u\left(t\right)\right)$ and in order to calculate derivative of $h$ with respect to $t$, we should use chain rule. The derivative is $$\dfrac{\partial h}{\partial t} = \dfrac{\partial h}{\partial u}\dfrac{\partial u}{\partial t}.$$

In general, if a function $h$ has several arguments, or even its argument is a vector, so that $h = h(x(t))$ where  $x \in R^n$ and $t \in R^m $. In this way, $h$ is now the combination of $n$ functions, each of which has $m$ variables. The derivative of $h$ is now

 $$\nabla_{t}h = \sum_{i=1}^{n}{\frac{\partial h}{\partial x_{i}}\nabla y_{i}\left(t\right)}.$$

### 2.2 Elementary functions

Any complex function is made up with several elementary functions. As discussed above, we use chain rule to break it down and then focus on elementary functions to calulate their derivatives. 

In mathematics, an elementary function is a function of a single variable composed of particular simple functions. Elementary functions are typically defined as a sum, product and/or composition of many polynomials, rational functions, trigonometric and exponential functions, and their inverse functions.<sup>5</sup>

On the other hand, we know the concrete mathematical expression of the elementary functions, which will be used directly in the later graph structure of calculations. 


### 2.3 Forward mode

#### 2.3.1 Evaluation trace

Take the example of $g = (x+y)*z$, we will first demonstrate the evaluation trace and then its corresponding computational graph.

Let's evaluate g at the point (1,1,1). In the evaluation trace table, we will record the trace of each step, its  elementary operation as well as the corresponding numeric value at the point.

| Trace | Elementary Operation | Numeric Value |
| ----- | -------------------- | ------------- |
| $x$   | 1                    | 1             |
| $y$   | 1                    | 1             |
| $z$   | 1                    | 1             |
| $p$   | $x+y$                | 2             |
| $f$   | $v_1*z$              | 2             |

#### 2.3.2 Computational graph

The above evaluation trace can be easily visualized with the computational graph below. The node will represent the trace and the edge will represent the elementary operation.

<img src="forward_mode.png">

*Note: If you cannot see the image, please right click and open image in new tab*

#### 2.3.3 Explanation
The evaluation trace above is just the path we will follow in forward mode. On top of that, we will also carry the derivatives. And we will take the derivative of g on x.

| Trace | Elementary Operation | Numeric Value | Deri. on x    | Deri. Value on x |
| ----- | -------------------- | ------------- | ------------- | ---------------- |
| $x$   | 1                    | 1             | 1             | 1                |
| $y$   | 1                    | 1             | 0             | 0                |
| $z$   | 1                    | 1             | 0             | 0                |
| $v_1$ | $x+y$                | 2             | $\dot{x}$     | 1                |
| $f$   | $v_1*z$              | 2             | $\dot{v_1}*z$ | 1                |

 ### 2.4 Dual Number
 
 A dual number has a real part and a dual part. Say we have $z = a + b\epsilon $. Then $a$ is the real part and $b$ is the dual part.

 For $\epsilon$, we define $\epsilon ^2 = 0$ but $\epsilon$ is not zero.
 
 Dual Number is really useful when we want to calculate derivatives of a function. For example, say we have

 $$ x = a + b\epsilon,   y = x^2$$
 
 Then we can derive
 
 $$y = (a + b\epsilon)^2 = a^2 + 2ab\epsilon + b^2\epsilon^2 = a^2 + 2ab\epsilon$$

 Therefore, it is really convenient to get the value of y from real part and get derivative of y from dual part. This is what we will implement in our FORWARD auto differentiation code.
 
 




## 3. Usage


 ### 3.1 Installation

There are two ways to install the `AutoDiff` package. For milestone 2, please see method 2. 
In the end, the package will be distributed through PyPI (Not yet implemented, a usage example is shown on demo.py).

* Method 1: install packaged using pip (This is a future implementation, because this time we have not yet uploaded our package to cloud)

    To install AutoDiff using pip, in the terminal, type:

    ```bash
    pip install AutoDiff
    ```

    This will also install all required documents as dependency.

    Run module tests before beginning,
    
    First, navigate to our package distribution website, download tar.gz folder, unzip, and enter the folder
    
    ```bash
    pytest tests
    ```
    Use AutoDiff package 
    
    ```python
    >>> import AutoDiff.autodiff as ad
    >>> from ad import *
    >>> import numpy as np
    #（begin autodifferentiation)
    >>> quit()
    ```


* Method 2: install using virtual environment
    
    For now, to get started, please do git clone on our project and test by using `demo.py`

    First, download the package from github to your folder

    ```bash
    mkdir test_merlionctc
    cd test_merlionctc
    git clone https://github.com/merlionctc/cs107-FinalProject.git
    cd cs107-FinalProject
    ```

    Create a vertual environment and activate it

    ```bash
    # If you do not have virtualenv, install it
    sudo easy_install virtualenv
    # Create virtual environment
    virtualenv ac207
    # Activate your virtual environmeny
    source ac207/bin/activate
    ```

    To ensure you have your enviroment and all required package setup, 

    ```bash
    pip install -r requirements.txt
    ```

    To install our package while using 
    
    ```bash
    pip install ./AutoDiff/dist/autodiff_merlionctc-0.0.1-py2.py3-none-any.whl
    ```

    To run a demo we provided.

    ```bash
    python3 AutoDiff/demo.py
    ```
    
    If you want to quit the virtual enviornment:

    ```bash
    deactivate
    ```
    

### 3.2 How to Use

Here is an example that serves that a quick start tutorial.

After installing AutoDiff package (see section 3.1 Method 2)

```python
python
>>> from autodiff.dual import *
>>> from autodiff.elementary import *
>>> from autodiff.model import *
>>> import numpy as np
```

First Step: User instantiate variables
* val: value of variable that you start with
* der: value of the derivative of variable that you start with, usually starting with 1
* loc: The location/index this variable when there are multiple input variables for the target function(s). For example, if you initialize x1 first, the loc will be 0; then you initialize y1, the loc will increment to 1
* length: The length/number of the total variables that will be input when there are multiple input variables for the target function(s).For example, if you want to initialize x1,y1 and z1, the length will be 3, for each variable in the initialization process

```python
>>>x1 = Dual(val = 1, der=1, loc = 0, length = 3)
>>>y1 = Dual(val = 2, der=1, loc = 1, length = 3)
>>>z1 = Dual(val = 5, der=1, loc = 2, length = 3)
```
Second Step: User inputs function, based on above variables
```python
>>>f1 = 3 * x1 + 4 * y1 * 2 - cos(z1)
```

Third Step: User instantiate `autodiff.Forward` class 
```python
>>>fwd_test = Forward(f1)
```

Fourth Step: User could choose to call instance method `get_value()` to get value of func
```python
>>>print(fwd_test.get_value())
18.716337814536775
```

Fifth Step: User could choose to call instance method `get_der()` to get derivatives of func

Note: This method will return a derivative vector w.r.t to ALL variables. 

Note 2: If user enters a scalar function, then get_der will return the jacobian

```python
>>>print(fwd_test.get_der())
[ 3.          8.         -0.95892427]
```

Sixth Step: User could choose to call instance method get_der(var) to get derivatives of func

Note: This method will return a derivative vector w.r.t to specific variables you input

```python
>>>print(fwd_test.get_der(x1))
[3.0]
```

quitting
```python
>>>quit()
```



## 4. Software Organization

Discuss how you plan on organizing your software package. We provided a basic structure here. 

* Directory Structure

```
AutoDiff
│   README.md
│   .travis.yml  
│   .coverage
│   requirements.txt
│
└───src
│   │
│   └─── autodiff
│   │   │   
│   │   └─── symbolic
│   │   │   │  expression.py (symbolic differentiation)
│   │   │
│   │   │   model.py (AutoDiff main class with Forward/Reverse mode)
│   │   │   elementary.py (Elementary Function module)
│   │   │   dual.py (Dual Number Class)
│   │   │   node.py (Node Class)
│        ...
│
└───tests
│   │   test_autodiff.py
│   │   test_dual_class.py
│   │   test_elementary.py
│   │   test_node_class.py
│   │   test_symbolic.py
│   │   ...
└───Docs
│   │   README.md
│   │   milestone1.ipynb
│   │   milestone2_progress.ipynb
│   │   milestone2.ipynb
│   │   documentation.ipynb
│   │   ...
│      
│   
└───demo.py (A demo to usage of package)

```

* **Modules to Include**

 Standrad Python libraries/modules:

 *math*: mathematical, algebric operations

 *numpy*: supports computations for large, multi-dimensional arrays and matrices. 

 Self-designed libraries/modules under `AutoDiff` package:

 *elementary*: contains overwritten elementary functions customized for the different variable class defined when implementing different types of differentiation.

 *model*: An interface-like, main class with Forward/Reverse mode auto-differentation methods. Also, the methods are useable for symbolic differentiation 

 *dual*: This module contains class methods for dual number class, which is the basic class structure in the forward mode

 *node*: This module contains class methods for node class, which is the basic class structure in the reverse mode

 *expression*: This module contains class methods and child classes of expression class, which is the basic class structure of symbolic differentiation


* **Test Suite Design**

 We used Pytest as our test suite. We tested all of our implemented classes on every single instance methods.
 We also tested on special and corner cases such as simplication, real number etc. 
 The whole test suite is included in the *test* sub-directory. 
 And both TravisCI and Coveralls will be used to check the codes coverage and test integration.

* **Package Distribution **

  * Package Distribution

    We will distribute our software using [Python Package Index (PyPi)](https://pypi.org/).
    Currently, we used Python project structure from [PyScaffold](https://pypi.org/project/PyScaffold/).

  * Software package

    We will package using the standard packaging tool ([setuptools](https://packaging.python.org/key_projects/#setuptools)), and following Package Python Projects [Tutorial](https://packaging.python.org/tutorials/packaging-projects/).

  As mentioned in 3.1 **_How to Install_**, the user can either choose to use `pip install` or installing using `github clone` to install the necessary dependencies.

  






## 5. Implementation

### 5.1 Forward Mode Implementation

The forward mode auto differentiation works for all real number functions.

#### 5.1.1 Core Data Structures

* The user will start by initializing variables using Dual class.

* Numpy Array:
  Our core data structure of Dual class and its operation is built upon numpy array.
  The elementary function is also operated on numpy array and real numbers.
  We store each variable its corresponding value, derivative value in numpy array, such that we could apply elementary operation to entire array instead of just a scalar value.
  
* For variables and differentiation point, it will be input in Dual number instantiation as its variable value, and value as the point of differentiation.
  User would also have to input the length of total variables and the position of current variable in dual class.

* For forward mode, the function is directly built upon operation on dual class initialized above.
  

#### 5.1.2 Implemented Classes

* `AutoDiff`: The base class for declaring a function that we wish to do auto-differenciation.

* `Forward(AutoDiff)`: Our forward auto-differentiation class. This class implements differention and calculate derivatives and jacobian through forward mode.
 
* `Dual`: dual number class. This class will take in any real number variable and construct and return it as a dual number,
  all the subsequent operations in AutoDiff will be done on Dual Number class.
  
  The class also contains operations (addition, subtraction, multiplication, division, power, etc) between dual numbers, and real numbers.
 

#### 5.1.3 Implemented Classes

`AutoDiff`:

* Description: This class declares a function to be differentiate. It provides a super method of differentiable function initialization for our Forward, Reverse, Symbolic differentiation classes. 

* Attributes: `self.f`: function to be differentiated, it shall be a list, or we initialize a user input as a list. 
We will pass this function into the child class of `AutoDiff`, which is, for example, `Forward`.

* Methods: 
  
  *  \__init__: intilization

`Forward(AutoDiff)`:

* Description: This class initialize a forward AD object, and contains method of getting values, derivatives, and jocobians

* Attributes: `self.f` is the function that we passed in, it is function of variables ready to be differentiated.

* Methods: 
  
  * \__init__: intilization, inherit from the super class
  * get_value: calculate the value of f on var through forward mode on specific value
  * get_der: calculate the derivative of f on var through forward mode with respect to all variables or specific directions
  * get_jacobian: calculate the jacobian matrix of f list on all vars through forward mode on all variables, user can specify the direction

`Dual`:

* Description: The Dual class supports custom operations for Automatic Differentiation (AD) in forward mode, it contains overwritten dunder method for basic operations
(addition, multiplication, exponentiation and so on)

* Attributes:

  `self.val`: The evaluation values of the functions or the initialized values for the variable(dual object)

  `self.der`: The derivatives of the functions or the initialized derivative of the variable. When there are multiple variables to be input in a function, the derivatives will be an
  array to store the derivatives of different variables separately.

  `self.loc`: The location/index of this variable when there are multiple input variables for the target function(s).

  `self.length`: The length/number of the total variables that will be input when there are multiple input variables for the target function(s).

* Methods:
 
  * \__init__: initialize a Dual object, with its values, derivatives, location and length.
  * \__repr__: Prints self in the form of Dual(value = [val], derivative = [der])
  * \__pos__: Returns the positive of self
  * \__neg__: Returns the negative of self
  * \__add__: Returns the addition of self and other, other can be Dual object, float, or int
  * \__radd__: Returns the addition of other and self, other can be Dual object, float, or int
  * \__sub__: Returns the subtraction of self and other, other can be Dual object, float, or int
  * \__rsub__:Returns the subtraction of other and self, other can be Dual object, float, or int
  * \__mul__: Returns the multiplication of self and other, other can be Dual object, float, or int
  * \__rmul__: Returns the multiplication of other and self, other can be Dual object, float, or int
  * \__truediv__: Returns the devision of self and other,other can be Dual object, float, or int
  * \__rtruediv__: Returns the devision of other and self, other can be float, or int
  * \__pow__: Returns the power of self raised by other, other can be Dual object, float, or int
  * \__rpow__: Returns the power of other raised by self, other can be float, or int 
  * \__eq__: Returns boolean if two objects have equal value 
  * \__ne__: Returns boolean if two objects DO NOT have equal value
  * \__lt__: Returns boolean if the former object is less than the latter
  * \__le__: Returns boolean if the former object is less than or equal to the latter
  * \__gt__: Returns boolean if the former object is greater than the latter
  * \__le__: Returns boolean if the former object is greater than or equal to the latter

### 5.2 Symbolic Reverse Mode

(For more detail, please onsult _6.1 symbolic differentiation_)

Expression class is an abstracted representation of a mathematical function. For instance, some expression represents functions in the form ``a+b``,
in addition to the arithematic expressions, there are two additional expressions, namly `Symbol` and `Constant`. 

Each expression knows how to evaluate itself against a list of given value of each `Symbol`.

Each expression are also implemented to differentiate themselves w.r.t any given `Symbol`, this differentiation returns an expression.

In this way, as each expression knows how to differentiate itself w.r.t a given `Symbol` and return an expression, which means each expression naturally has the ability of doing higher order function.

To get value, derivative or jacobian, we just needs to evaluate expression, derivative expression or jacobian expression at their given value.

Conceptually, by maintaining a syntax tree, this implementation also meets our definition of `reverse mode differentiation` in the sense that each expression asks its dependencies, combining the results of its dependencies only at the time when `diff()` is called.


#### 5.2.1 Core Data Structures


* `Expression` class:
  Our core data structure of will be the Expression class its operations are self-contained.

* Dictionary: the values that the expression will be evaluated against will be entered as a dictionary.
  
* NumPy array: We store each variable its corresponding value, derivative value in numpy array, such that we could apply elementary operation to entire array instead of just a scalar value.

For variables and differentiation point, it will be input in `Symbol` instantiation as its variable value as an dictionary. For symbolic differentiation, the function is directly built upon operation on Expression class initialized above.
  

#### 5.2.2 Implemented Classes

* `Expression`: The parent class for declaring a Expression object (function) that we wish to do auto-differenciation.

* `Constant(Expression)`: Our forward auto-differentiation class. This class implements differention and calculate derivatives and jacobian through forward mode.
 
* `Symbol(Expression)`: dual number class. This class will take in any real number variable and construct and return it as a dual number,
  all the subsequent operations in AutoDiff will be done on Dual Number class.
  
  The class also contains operations (addition, subtraction, multiplication, division, power, etc) between dual numbers, and real numbers.
 
* Operation specific classes: basic arithmatic operation, turns into an sub-class of `Expression`. Including `SumExpression(Expression)`,`ProductExpression`,
`DivisionExpression(Expression)`

#### 5.1.3 Implemented Classes

`AutoDiff`:

* Description: This class declares a function to be differentiate. It provides a super method of differentiable function initialization for our Forward, Reverse, Symbolic differentiation classes. 

* Attributes: `self.f`: function to be differentiated, it shall be a list, or we initialize a user input as a list. 
We will pass this function into the child class of `AutoDiff`, which is, for example, `Forward`.

* Methods: 
  
  *  \__init__: intilization




  * **Elementary Function module** 
  
 For the elementary function, we write our own method of computing the value so that these function can be applied on the Dual number, as well as on the real number. 
 For example, in our daily usage, sine funtion on a real number `x` can be calculated via `NumPy` (np.sin(x)), but here when we calculate sine value for a Dual object, 
 we cleverly store the value (same as we got from np.sin(x)) and the derivative part. 

 We have implemented the following elementary functions and we provide a demo function for reference.
 
```python
#the function we have implemented so far, * represents the input

# Exponentials
exp(*): #extend the exponential function to Dual number

# Square root
sqrt(*): #extend the square root function with base e to Dual number

# Trig functions
sin(*): #extend the sine function with base e to Dual number
cos(*): #extend the cosine function with base e to Dual number
tan(*): #extend the tangent function with base e to Dual number

# Inverse trig functions
arcsin(*): #extend the inverse sine function with base e to Dual number
arccos(*): #extend the inverse cosine function with base e to Dual number
arctan(*): #extend the inverse tangent function with base e to Dual number

# Hyperbolic functions
sinh(*)): #extend the hyperbolic sine function with base e to Dual number
cosh(*): #extend the hyperbolic cosine function with base e to Dual number
tanh(*): #extend the hyperbolic tangent function with base e to Dual number

# Logistic function
logistic(*): #extend the logistic function with base e to Dual number, the default is a standard sigmoid

#Logarithms
log(*): #extend the natural log to Dual number
logb(*,base): #extend the log function with any base to Dual number

```

Here is the demo:

```python
def sin(dual):
    """Calculate the sine of the input
        Keyword arguments:
        dual -- a dual number or a real number
        Return:
        the sine value
    """
    if isinstance(dual, Dual):
        der = np.cos(dual.val)*dual.der
        val = np.sin(dual.val)
        return Dual(val,der)
    else:
        return np.sin(dual)
             
```
If we call the above function, it will give the following output. 

```python
#...import necessary dependencies
x = Dual(np.pi, 1)
z = sin(x)
print(z)
```
We will get:
```python
Dual(value=1.2246467991473532e-16, derivative=-1.0)

```

Notice z is a Dual object. This function also applies to real number:

```python
#...import necessary dependencies
x_real = np.pi
z_real = sin(x_real)
print(z_real)


```
We will get:
```python
1.2246467991473532e-16
```

In the future, we may consider extending our elementary function libraries, including the hypobolic sine and cosine, as well as functions like csc and cot, log with base 2 and 10, if needed.

* **External Dependencies**

We have the following external libraries/Modules to include:

`NumPy`: This provides an API for a large collection of high-level mathematical operations. In addition, it provides support for array operations.

`Math`: This provides access to some mathematical functions also.

`pytest`: This is the way we perform testing on our codes.

Also, we include Travis CI and CodeCov to make sure that the validility of building and the code. 
  
* **Future implementation**

Please refer to section 6. Future Feature.




## 6. Extensions: Reverse Mode & Symbolic Expression

### 6.1 Symbolic Expression (Reverse Diff and Higher Order Derivative) 
For extension, we built a Expression class to implement the symbolic representation of our function. 

#### 6.1.1 Implementation
Expression class is an abstracted representation of a mathematical function. For instance, some expression represents functions in the form ``a+b``,
in addition to the arithematic expressions, there are two additional expressions, namly `Symbol` and `Constant`. 

Each expression knows how to evaluate itself against a list of given value of each `Symbol`.

Each expression are also implemented to differentiate themselves w.r.t any given `Symbol`, this differentiation returns an expression.

In this way, as each expression knows how to differentiate itself w.r.t a given `Symbol` and return an expression, which means each expression naturally has the ability of doing higher order function.

To get value, derivative or jacobian, we just needs to evaluate expression, derivative expression or jacobian expression at their given value.

Conceptually, by maintaining a syntax tree, this implementation also meets our definition of `reverse mode differentiation` in the sense that each expression asks its dependencies, combining the results of its dependencies only at the time when `diff()` is called.


#### 6.1.2 Functionality
* Evaluate 

* Differentation

* Evaluate at Differentiation Expression

* Higher Order Differentation


#### 6.1.3 Usage




### 6.2 Reverse Mode

#### 6.2.1 Explanation and Background

It should be noticed that in forward mode, chain rule is not utilized. We just follow the evaluation trace and combine the derivatives of elementary functions together. But for reverse mode, we will implement chain rule. 

However, it is important to realize that the reverse mode also requires the evaluation trace on forward mode to have the derivatives on the elementary functions. Then we will use chain rule to reversely calculate the final derivative.

The steps we use to implement reverse mode based on the evaluation trace in forward mode is as follows.

- STEP1: Start with $v_1$

   $$\overline{v_1} = \dfrac{\partial f}{\partial v_1} = 1.$$

- STEP2: Use chain rule to calculate$\overline{x}$

   $$\overline{x} = \dfrac{\partial f}{\partial v_1}\dfrac{\partial v_1}{\partial x}  = 1.$$

- STEP3: Get the derivative on x

   $$\overline{x} = \dfrac{\partial f}{\partial x}  = 1.$$

** Computational graph **

In reverse mode, we just reversely implement chain rule to get the derivatives. And the computational graph for reverse mode is as follows.

<img src="reverse_mode.png">

*Note: If you cannot see the image, please right click and open image in new tab*

#### 6.2.2 Implementation
To implement reverse mode, we create a Node Class to store the nodes/traces in the computational graph, modify elementary functions to customize node objects,

to store the value, input nodes, gradients with respect to the input nodes, and its derivative.


## 7. Broader Impact and Inclusivity Statement 

### Broader Impact



### Software Inclusivity
This package welcomes and encourage participation and usage from a global community. 
Just as Python Software's Diversity Statement indicated, *the Python community is based on mutual respect, tolerance, and encouragement, and we are working to help each other live up to these principles.* We, as the developer of this AutoDiff package, also want our user group to be more diverse: whoever you are, and whatever your background, we welcome you to use our package.
This software package is built based upon the diversity perspective on Python broader community. We strongly believe that embrace diverse community to use our package brings new blood and perspective, making our user group stronger and more vibrant. A diverse user group where all users treat each other with respect has more potential contributors and more sources for fresh ideas.
We also welcomes users from all language background. Mathematics has no boundary.


## 8. Reference

[[1]](https://www.jmlr.org/papers/volume18/17-468/17-468.pdf): Baydin, Atilim Gunes; Pearlmutter, Barak; Radul, Alexey Andreyevich; Siskind, Jeffrey (2018). "Automatic differentiation in machine learning: a survey". Journal of Machine Learning Research. 18: 1–43.

[[2]](https://fac.ksu.edu.sa/sites/default/files/numerical_analysis_9th.pdf):Rirchard L. Burden and J. Douglas Faires. Numerical Analysis. Brooks/Cole, 2001.

[[3]](https://www.springer.com/gp/book/9783540654667):Johannes Grabmeier and Erich Kaltofen. Computer Algebra Handbook: Foundations, Applications, Systems. Springer, 2003

[[4]](https://www.jstor.org/stable/24103956): Arun Verma. An introduction to automatic differentiation. Current Science, 78(7):804–7,
2000.

[[5]](https://www.worldcat.org/oclc/31441929):  Spivak, Michael. (1994). *Calculus* (3rd ed.). Houston, Tex.: Publish or Perish. p. 359.


